In [3]:
import math

def generate_non_linear_dial(num_major=5, num_minor=5, full_scale=300, dial_shape='quarter', unit='A', width=300, height=300, accuracy = 1.5, output_file="controlled_non_linear_dial.svg"):
    """Generates a dial SVG file with controlled non-linear tick spacing based on a specified non-linearity factor."""
    
    # SVG header
    svg_header = f'<svg xmlns="http://www.w3.org/2000/svg" width="{width}" height="{height}" viewBox="0 0 {width} {height}">\n'
    
    # Center point based on shape
    if dial_shape == 'quarter':
        center_x, center_y = width - 60, height - 40
    elif dial_shape == 'semi':
        center_x, center_y = (width // 2) - 60, height - 40
    elif dial_shape == 'full':
        center_x, center_y = (width // 2) - 60, (height // 2) - 40
    else:
        raise ValueError("Dial shape must be 'quarter', 'semi', or 'full'")
    
    radius = min(width, height) - 100  # Adjust radius to fit within the given dimensions
    
    #non_linearity = 0.6 * (600 / width)
    non_linearity = math.log(radius / 100 + 1)
    
    # Major and minor tick lengths
    major_tick_length = (width * 0.1)
    thin_line_length = major_tick_length / 2
    thick_line_length = major_tick_length / 2
    minor_tick_length = (width * 0.05) - 10 if (width * 0.05) - 10 > 10 else 20
    
    # Define total angle for different dial shapes
    if dial_shape == 'quarter':
        total_angle = 90
    elif dial_shape == 'semi':
        total_angle = 180
    elif dial_shape == 'full':
        total_angle = 360
    else:
        raise ValueError("Dial shape must be 'quarter', 'semi', or 'full'")
    
    # SVG content string
    svg_content = ''

    # Function to calculate coordinates on the dial based on angle
    def calculate_coords(angle_degrees, radius):
        angle_radians = math.radians(270 - angle_degrees)
        x = center_x + radius * math.cos(angle_radians)
        y = center_y + radius * math.sin(angle_radians)
        return x, y
    
    # Generate non-linear positions for major ticks using an exponential function
    for i in range(num_major):
        # Adjust angle using non-linearity factor, which controls the exponent for spacing
        angle = total_angle * ((i / (num_major - 1)) ** non_linearity)

        # Thin line segment for major tick
        x1, y1 = calculate_coords(angle, radius)
        x2, y2 = calculate_coords(angle, radius - thin_line_length)
        svg_content += f'<line x1="{x1:.2f}" y1="{y1:.2f}" x2="{x2:.2f}" y2="{y2:.2f}" stroke="black" stroke-width="{0.005 * width}"/>\n'
        
        # Thick line segment for major tick
        x3, y3 = calculate_coords(angle, radius - thin_line_length)
        x4, y4 = calculate_coords(angle, radius - major_tick_length)
        svg_content += f'<line x1="{x3:.2f}" y1="{y3:.2f}" x2="{x4:.2f}" y2="{y4:.2f}" stroke="black" stroke-width="{0.01 * width}"/>\n'

        # Label for major tick
        label_value = int(full_scale - (i * (full_scale / (num_major - 1))))
        label_x, label_y = calculate_coords(angle, radius - major_tick_length - (width * 0.08))
        svg_content += f'<text x="{label_x:.2f}" y="{label_y:.2f}" font-size="{0.08 * width}" text-anchor="middle" alignment-baseline="middle">{label_value}</text>\n'
        

    # Generate minor tick positions
    for i in range(num_major - 1):
        angle_start = total_angle * ((i / (num_major - 1)) ** non_linearity)
        angle_end = total_angle * (((i + 1) / (num_major - 1)) ** non_linearity)
        minor_angle_step = (angle_end - angle_start) / (num_minor + 1)
        
        for j in range(1, num_minor + 1):
            angle = angle_start + j * minor_angle_step
            
            # Minor tick start and end coordinates
            x1, y1 = calculate_coords(angle, radius)
            x2, y2 = calculate_coords(angle, radius - minor_tick_length if not(j%2) else radius - (minor_tick_length - 10))
            
            # Draw minor tick line
            svg_content += f'<line x1="{x1:.2f}" y1="{y1:.2f}" x2="{x2:.2f}" y2="{y2:.2f}" stroke="black" stroke-width="{0.005 * width}"/>\n'
    
    # Place unit at the top left corner
    svg_content += f'<text x="{height * 0.25}" y="{width * 0.25}" font-size="{0.1 * width}">{unit}</text>\n'

    # Add metrics below the dial side by side
    metrics_y = height - 20
    metrics_x = 60
    svg_content += f'<text x="{metrics_x}" y="{metrics_y + 20}" font-size="{0.05 * width}">{accuracy}</text>\n'
    svg_content += f'<text x="{metrics_x + 40}" y="{metrics_y + 20}" font-size="{0.05 * width}">{1.5}</text>\n'
    svg_content += f'<text x="{metrics_x + 80}" y="{metrics_y + 20}" font-size="{0.05 * width}">{"150/5A"}</text>\n'
    
    # SVG footer
    svg_footer = '</svg>'
    
    # Write to SVG file
    with open(output_file, 'w') as f:
        f.write(svg_header)
        f.write(svg_content)
        f.write(svg_footer)

    print(f"Dial saved to {output_file} with size {width}x{height}")

# Example usage with non-linearity control
generate_non_linear_dial()

Dial saved to controlled_non_linear_dial.svg with size 300x300
